# HIGT Usage Tutorial:

**HIGT** uses the 512 $\times$ 512 size patches segmented from the WSIs at 5x and 10x magnification and thumbnail as the input data. In order to allow readers to better understand and practice, below we provide a complete usage tutorial.

You can download the [TCGA data](https://portal.gdc.cancer.gov/) from this link. And this tutorial is divided into three main parts: Preprocessing, GPU Training, Testing, and Evaluation and is mainly built on [CLAM](https://github.com/mahmoodlab/CLAM), but we changed some codes to suit the specific needs. The details of them are as follows:

## 1. Preprocessing:
This section is mainly divided into five parts: Basic Information Statistics, Patch Segmentation, Feature Extraction, Hierarchical Graph (Tree) Generation and Dataset Split. The details of each part are as follows:

### 1.1 Basic Information Statistics:

Use the `generate_pl_bm` function to analyze the basic information of the WSIs. It mainly realizes the following three functions:

1. **Objective Lens Magnification**: 
Count and record the target WSI file's default objective lens magnification information. The recorded data is saved in a file named `bm.csv`. The structure and content of it are shown in the table below:

<center>

| Column Name | Description | Data Type |
|-------------|-------------|-----------|
| slide_path | Saving path of WSI files | String |
| base_mag | Default magnification of WSI files | String |

</center>

2. **Process List Generation**: 
Based on the target objective lens magnification and the cutting block size of the target objective lens magnification, generate the corresponding `pl_mag{target_magnification}x_patch{base_patch_size}_{target_patch_size}.csv`. The structure and content of it are shown in the table below:

<center>

| Column Name | Description | Data Type |
|-------------|-------------|-----------|
| slide_id | File name of the WSI files | String |

</center>

3. **Data cleaning**: 
Cleared some WSIs without default objective magnifications.

##### (1) Parameter Description:

- **`--WSI_dir`**: Saving directory of the WSI files.
- **`--save_dir`**: Saving directory of the generated CSV files.
- **`--base_patch_size`**: Patch size at the target magnification.
- **`--target_mag`**: Target magnification.

##### (2) Usage Example:

In the following experiments, we use a dataset with a default 40x magnification to demonstrate, called `WSI_bm40`, and the WSI files of `WSI_bm40` are saved in the `/path/to/exp/WSI_bm40/WSI_bm40` directory. The codes for basic statistics can be structured as follows:

```python
generate_pl_bm(
        WSI_dir="/path/to/exp/WSI_bm40/WSI_bm40", 
        save_dir="/path/to/exp/WSI_bm40/csv", 
        base_patch_size=512, 
        target_mag=5
)

generate_pl_bm(
        WSI_dir="/path/to/exp/WSI_bm40/WSI_bm40", 
        save_dir="/path/to/exp/WSI_bm40/csv", 
        base_patch_size=512, 
        target_mag=10
)
```

**Note**: Please replace the paths and values with those relevant to your setup.

In [1]:
def generate_pl_bm(
        WSI_dir, 
        save_dir, 
        base_patch_size, 
        target_mag
    ):
    import openslide, glob
    import pandas as pd

    process_list = {} 
    base_mag_csv = {
        "slide_path": [],
        "base_mag": []
    }

    for WSI in glob.glob(WSI_dir+"/*"):
        slide = openslide.open_slide(WSI)
        wsi_name = WSI.split("/")[-1]
        if slide.properties.get(openslide.PROPERTY_NAME_OBJECTIVE_POWER) == None:
            continue

        base_mag = int(slide.properties.get(openslide.PROPERTY_NAME_OBJECTIVE_POWER))
        target_min_patch_size = int(base_patch_size*(base_mag/target_mag))

        # Update for process_list
        if target_min_patch_size not in process_list:
            process_list[target_min_patch_size] = [wsi_name]
        else:
            process_list[target_min_patch_size].append(wsi_name)

        # Update for bm
        base_mag_csv["slide_path"].append(WSI)
        base_mag_csv["base_mag"].append(base_mag)

        # save bm.csv
        df = pd.DataFrame(base_mag_csv)
        df.to_csv(save_dir+f"bm.csv")

        # save patch_size_i process_list.csv
        for k in process_list.keys():
            df = pd.DataFrame({
                "slide_id": process_list[k]
            })
            df.to_csv(save_dir+f"pl_mag{target_mag}x_patch{base_patch_size}_{k}.csv")

### 1.2 Patch Segmentation:
Patch segmentation is required to be performed based on the default magnification of the WSI files and the patch size used at target magnification, called `target_patch_size`. Finally, the x and y coordinates of the upper left corner of the patch segmentation result can be obtained. 

For the **HIGT**, the segmentation results at 5x and 10x magnification with a patch size of 512 are required. First the the segmentation process at 5x magnification is performed based on CLAM's `create_patches_fp.py`, and then the cutting results at 10x magnification of the corresponding data set are generated according to the cutting results at 5x magnification and the `generate_coords_file` function below.

#### 1.2.1 Patch segmentation at 5x magnification:

##### (1) Parameter Description:

- **`--source`**: Saving directory of the WSI files.
- **`--save_dir`**: Saving directory of the patch segmentation results.
- **`--patch_size`**: Patch size at target magnification.
- **`--step_size`**: Step size for segmentation. If no overlap is required, this should be equal to the `patch_size`.
- **`--seg`**: Flag to indicate whether to generate the mask.
- **`--patch`**: Flag to indicate whether to generate the patch.
- **`--stitch`**: Flag to indicate whether to generate the stitch.
- **`--process_list`**: Path of the ßprocess list file, mainly utilizing the `slide_id` column, and other columns can be omitted.

##### (2) Usage Example:
The command line for patch segmentation can be structured as follows:

```bash
python create_patches_fp.py --source /path/to/exp/WSI_bm40/WSI_bm40 --save_dir /path/to/exp/WSI_bm40/segmented_patch/mag5x_patch512_4096 --patch_size 4096 --step_size 4096 --seg --patch --stitch --process_list /path/to/exp/WSI_bm40/csv/pl_mag5x_patch512_4096.csv
```

**Note**: Please replace the paths and values with those relevant to your setup.

#### 1.2.2 Generation of patch segmentation at 10x magnification:

Use the `generate_coords_file` function below to generate the segmentation results at 10x magnification based on the segmentation results at 5x magnification.

##### (1) Parameter Description:
- **`--source_mag_seg_dir`**: Saving directory of the segmentation result generated at last step.
- **`--target_mag`**: Target magnification.

##### (2) Usage Example:

The code for the generation of the patch segmentation can be structured as follows:

```python
generate_coords_file(
    source_mag_seg_path="/path/to/exp/WSI_bm40/segmented_patch/mag5x_patch512_4096", 
    target_mag=10
)
```

**Note**: Please replace the paths and values with those relevant to your specific setup.

In [ ]:
def generate_coords_file(source_mag_seg_path, target_mag):
    
    import os, h5py, glob
    import numpy as np
    from wsi_core.wsi_utils import save_hdf5

    cur_mag = int(source_mag_seg_path.split("/")[-1].split("_")[0].replace("mag","").replace("x",""))
    cur_patch_size = int(source_mag_seg_path.split("/")[-1].split("_")[-1])
    target_patch_size = int(cur_patch_size/int(target_mag/cur_mag))

    for h5 in glob.glob(source_mag_seg_path+"/patches/*"):
        h5_content = h5py.File(h5,'r')

        coords = h5_content["coords"][:]
        save_path = "/".join(source_mag_seg_path.split("/")[:-1])+f"/mag{target_mag}x_patch512_{target_patch_size}/patches/"

        if not os.path.exists(save_path):
            os.makedirs(save_path)

        attr = {
            'patch_size' :            target_patch_size, 
            'patch_level' :           h5_content["coords"].attrs["patch_level"],
            'downsample':             h5_content["coords"].attrs["downsample"],
            'downsampled_level_dim' : h5_content["coords"].attrs["downsampled_level_dim"],
            'level_dim':              h5_content["coords"].attrs["level_dim"],
            'name':                   h5_content["coords"].attrs["name"],
            'save_path':              save_path
        }

        h5_content.close()
        coords_ = []
        for coord in coords:
            x,y = coord
            coords_.append([x,y])
            coords_.append([x+target_patch_size,y])
            coords_.append([x,y+target_patch_size])
            coords_.append([x+target_patch_size,y+target_patch_size])
            
        coords_ = np.array(coords_).astype(coords.dtype)
        unique_coords = np.unique(coords_, axis=0)

        save_hdf5(
            save_path+h5.split("/")[-1], 
            {"coords": unique_coords}, 
            {"coords":attr}, 
            mode="w"
        )

        # break

### 1.3 Feature Extraction:

For the feature extraction at 5x and 10x magnification, based on CLAM's `extract_features_fp.py`, some changes have been made, the feature shape extracted from each patch image can be selected, re-write the `extract_features_fp_re.py` file. The feature extraction of the thumbnail of WSI can be implemented using the following `extract_features_thumb` function.

#### 1.3.1 Feature Extraction at 5x and 10x magnification:

##### (1) Parameter Description:

- **`--csv_path`**: Path of the process list file, mainly utilizing the `slide_id` column, and other columns can be omitted.
- **`--data_h5_dir`**: Saving directory of the patch segmentation results.
- **`--data_slide_dir`**: Saving directory of the WSI files.
- **`--feat_dir`**: Saving directory of the extracted features.
- **`--batch_size`**: Batch size (e.g., `32`, `64`, etc.).
- **`--target_patch_size`**: Size of the input image to the encoder. There is a built-in downsampling function.
- **`--slide_ext`**: Suffix for WSI files.
- **`--out_shape`**: Shape of the extracted feature.

##### (2) Usage Example:

The command line for feature extraction can be structured as follows:

```bash
# 5x Magnification
python extract_features_fp_re.py --csv_path /path/to/WSI_bm40/csv/pl_mag5x_patch512_4096.csv --data_h5_dir /path/to/exp/WSI_bm40/segmented_patch/mag5x_patch512_4096 --data_slide_dir /path/to/exp/WSI_bm40/WSI_bm40 --feat_dir /path/to/exp/WSI_bm40/extracted_feature/mag5x_patch512_4096 --batch_size 256 --target_patch_size 4096 --slide_ext .svs --out_shape 1024

# 10x Magnification
python extract_features_fp_re.py --csv_path /path/to/WSI_bm40/csv/pl_mag10x_patch512_2048.csv --data_h5_dir /path/to/exp/WSI_bm40/segmented_patch/mag10x_patch512_2048 --data_slide_dir /path/to/exp/WSI_bm40/WSI_bm40 --feat_dir /path/to/exp/WSI_bm40/extracted_feature/mag10x_patch512_2048 --batch_size 256 --target_patch_size 2048 --slide_ext .svs --out_shape 1024
```

**Note**: Please replace the paths and values with those relevant to your setup.

### 1.3.2 Feature Extraction for Thumbnail:

Based on the `extract_features_thumb` function, extract features from WSI thumbnail images.

##### (1)Parameter Description:
- **`--data_slide_dir`**: Saving directory of the WSI files.
- **`--cuda_id`**: ID of GPUs to use.
- **`--feat_dir`**: Saving directory of the extracted features.

##### (2) Usage Example:

The code for feature extraction of thumbnail can be structured as follows:

```python
feature_extract_thumb(
    data_slide_dir="/path/to/exp/WSI_bm40/WSI_bm40",
    cuda_id=0,
    feat_dir="/path/to/exp/WSI_bm40/extracted_feature"
)
```

**Note**: Please replace the paths and values with those relevant to your specific setup.

In [1]:
def feature_extract_thumb(
    data_slide_dir,
    cuda_id,
    csv_path,
    feat_dir
):
    import openslide, glob, torch, h5py, os
    from torchvision import transforms
    from models.resnet_custom import resnet50_baseline
    import pandas as pd

    df = pd.read_csv(csv_path)
    slide_ids = df["slide_id"]

    device = torch.device(f"cuda:{cuda_id}" if torch.cuda.is_available() else "cpu")
    
    model = resnet50_baseline(pretrained=True).to(device)
    model.eval()

    transform = transforms.Compose([
        transforms.ToTensor()
    ])
    with torch.no_grad():
        for slide_id in slide_ids:
            slide_path = os.path.join(data_slide_dir,slide_id)
            slide = openslide.OpenSlide(slide_path)
            thumbnail_image = slide.get_thumbnail((512,512))
            thumbnail_tensor = transform(thumbnail_image).to(device)

            thumbnail_feat = model(thumbnail_tensor.unsqueeze(0))
            out_root_path = f'{feat_dir}/thumbnail/'
            if not os.path.exists(out_root_path):
                os.makedirs(out_root_path)
            with h5py.File(os.path.join(out_root_path,"h5_files",slide_path.split("/")[-1]+".h5"), 'w') as h5f:
                h5f.create_dataset('features', data=thumbnail_feat.cpu().numpy())
                h5f.create_dataset('coords', data=[[0,0]])
            print(slide_path.split("/")[-1]," done")

### 1.4 Graph (Tree) Generation:
Based on the above extracted features at 5x and 10x magnification and thumbnail, use the `generate_graph_tree()` function to construct a input graph (tree) data. 

##### (1) Parameter Description:
- **`--target_mags`**: Target magnifications.
- **`--bm_path`**: Path of the base magnification information file.
- **`--feature_dir`**: Saving directory of the extracted features.
- **`--base_patch_size`**: Patch size at the target magnification.
- **`--save_path`**: Path of the generated graphs.

##### (2) Usage Example:

The code for graph generation can be structured as follows:

```python
generate_graph_tree(
     target_mags = [float("inf"), 5, 10],
     bm_path = "/path/to/exp/WSI_bm40/csv/bm.csv",
     feature_dir = "/path/to/exp/WSI_bm40/extracted_feature/",
     base_patch_size = 512,
     save_path = "/path/to/exp/WSI_bm40/higt_graph/"
)
```

**Note**: Please replace the paths and values with those relevant to your setup.

In [10]:
def generate_graph_tree(
    target_mags, bm_path, feature_dir, base_patch_size, save_path
):
    import os, h5py, torch
    from torch_geometric.data import Data
    import numpy as np
    import pandas as pd
    def get_edge_index(feature_i, region_patch_size, patch_patch_size):
    
        start = []
        end = []

        len_thumbnail = 1
    
        region_mag = list(feature_i.keys())[1]
        region_features = feature_i[region_mag]["features"]
        region_coords = feature_i[region_mag]["coords"]
        len_region_node = len(region_features)

        patch_mag = list(feature_i.keys())[2]
        patch_features = feature_i[patch_mag]["features"]
        patch_coords = feature_i[patch_mag]["coords"]
        len_patch_node = len(patch_features)
        
        # 1. region_level
        for i in range(len_region_node):

            # 1. thumbnail <=> region
            region_index = i+len_thumbnail
            x,y = region_coords[i]
            start.append(0)
            end.append(region_index)
            start.append(region_index)
            end.append(0)

            # 2. region => region
            nb_coord_list = [
                [x-region_patch_size, y-region_patch_size], [x, y-region_patch_size], [x+region_patch_size, y-region_patch_size],
                [x-region_patch_size, y], [x+region_patch_size, y],
                [x-region_patch_size, y+region_patch_size], [x, y+region_patch_size], [x+region_patch_size, y+region_patch_size]
            ]
            for xy_ in nb_coord_list:
                x_, y_ = xy_
                if np.any(np.all(region_coords == [x_, y_], axis=1)):
                    to_region_index = np.where((region_coords==(x_, y_)).all(axis=1))[0][0]+len_thumbnail
                    start.append(region_index)
                    end.append(to_region_index)
                    start.append(to_region_index)
                    end.append(region_index)

        # 2. patch_level
        for i in range(len_patch_node):
            patch_index = i+len_thumbnail+len_region_node
            x,y = patch_coords[i]

            # 1. patch <=> region
            region_coord_list = [[x,y]]
            if x-patch_patch_size>0:
                region_coord_list.append([x-patch_patch_size, y])
            elif y-patch_patch_size>0:
                region_coord_list.append([x, y-patch_patch_size])
            elif x-patch_patch_size>0 and y-patch_patch_size>0:
                region_coord_list.append([x-patch_patch_size, y-patch_patch_size])

            for xy_ in region_coord_list:
                x_,y_ = xy_
                if np.any(np.all(region_coords == [x_, y_], axis=1)):
                    region_index = np.where((region_coords==(x_,y_)).all(axis=1))[0][0]+len_thumbnail
                    start.append(patch_index)
                    end.append(region_index)
                    start.append(region_index)
                    end.append(patch_index)

            # 2. patch <=> patch
            nb_coord_list = [
                [x-patch_patch_size, y-patch_patch_size], [x, y-patch_patch_size], [x+patch_patch_size, y-patch_patch_size],
                [x-patch_patch_size, y], [x+patch_patch_size, y],
                [x-patch_patch_size, y+patch_patch_size], [x, y+patch_patch_size], [x+patch_patch_size, y+patch_patch_size]
            ]
            for xy_ in nb_coord_list:
                x_,y_ = xy_
                if np.any(np.all(patch_coords == [x_, y_], axis=1)):
                    to_patch_index = np.where((patch_coords==(x_,y_)).all(axis=1))[0][0]+len_thumbnail+len_region_node
                    start.append(patch_index)
                    end.append(to_patch_index)
                    start.append(to_patch_index)
                    end.append(patch_index)
        return [start, end]

    all_data = {}
    bm = pd.read_csv(bm_path)
    
    for slide_path,base_mag in zip(bm['slide_path'],bm['base_mag']):
        wsi_name = slide_path.split("/")[-1].replace(".svs","")
        h5_name = wsi_name+".h5"
        
        feature_i = {}
        all_feature = []

        min_mag = min(target_mags)
        min_patch_size = int(base_patch_size*(base_mag/min_mag))

        for cur_mag in target_mags:
            if cur_mag!=float("inf"):
                cur_patch_size = int(base_patch_size*(base_mag/cur_mag))
                cur_feature_path = os.path.join(feature_dir,f"mag{cur_mag}x_patch512_{cur_patch_size}/h5_files/{h5_name}")
            else:
                cur_feature_path = os.path.join(feature_dir,f"thumbnail/h5_files/{h5_name}")
            h5_content = h5py.File(cur_feature_path,'r')
            h5_features = h5_content["/features"][:]
            h5_coords = h5_content["/coords"][:]
            if cur_mag == float("inf"):
                cur_mag = 0
            feature_i[cur_mag] = {
                "features": h5_features,
                "coords": h5_coords
            }
            if len(h5_features.shape)<2:
                h5_features = np.expand_dims(h5_features, 0)
            all_feature.append(h5_features)

        # generate the Data
        # 0. preparation
        region_mag = list(feature_i.keys())[1]
        patch_mag = list(feature_i.keys())[2]

        region_coords = feature_i[region_mag]["coords"]
        patch_coords = feature_i[patch_mag]["coords"]


        len_region_node, len_patch_node = len(region_coords), len(patch_coords)

        region_patch_size = int((base_mag/region_mag)*base_patch_size)
        patch_patch_size = int((base_mag/patch_mag)*base_patch_size)

        # 1. x
        all_feature = np.concatenate(all_feature, axis=0)

        # 2. edge_index_tree_8nb
        edge_index = get_edge_index(feature_i, region_patch_size, patch_patch_size)

        # 3. batch
        batch = [0]*len(all_feature)

        # 4. data_id
        data_id = wsi_name.split(".")[0]

        # 5. node_type
        node_type = [0]+[1]*len_region_node+[2]*len_patch_node

        # 6. node_tree
        node_tree_wo_patch = [-1]+[0]*len_region_node
        node_tree_patch = []
        for i in range(len_patch_node):
            x, y = patch_coords[i]

            region_coord_list = [[x,y]]
            if x-patch_patch_size>0:
                region_coord_list.append([x-patch_patch_size, y])
            elif y-patch_patch_size>0:
                region_coord_list.append([x, y-patch_patch_size])
            elif x-patch_patch_size>0 and y-patch_patch_size>0:
                region_coord_list.append([x-patch_patch_size, y-patch_patch_size])

            for xy_ in region_coord_list:
                x_,y_ = xy_
                if np.any(np.all(region_coords == [x_, y_], axis=1)):
                    region_index = np.where((region_coords==(x_,y_)).all(axis=1))[0][0]+1
            node_tree_patch.append(region_index)
        node_tree = node_tree_wo_patch+node_tree_patch

        # 7. x_y_index 
        region_patch_size = int(base_patch_size*(base_mag/target_mags[1]))
        patch_patch_size = int(base_patch_size*(base_mag/target_mags[2]))
        # print(np.array([[0,0]]).shape, region_coords.shape, patch_coords/patch_patch_size)
    
        x_y_index = np.concatenate([
            np.array([[0,0]]),
            region_coords/region_patch_size,
            patch_coords/patch_patch_size,
        ])
    
        # generate the Data 
        node_attr=torch.tensor(all_feature, dtype=torch.float)
        edge_index_tree_8nb = torch.tensor(edge_index,dtype=torch.long)
        batch = torch.tensor(batch)
        node_type = torch.tensor(node_type)
        node_tree = torch.tensor(node_tree)
        x_y_index = torch.tensor(x_y_index, dtype=torch.float)

        data = Data(
            x = node_attr,
            edge_index_tree_8nb = edge_index_tree_8nb,
            data_id = data_id,
            batch = batch,
            node_type = node_type,
            node_tree = node_tree,
            x_y_index = x_y_index
        )
        if not os.path.exists(os.path.join(save_path, "pt_files")):
            os.makedirs(os.path.join(save_path, "pt_files"))

        output_path = os.path.join(save_path, "pt_files", wsi_name)
        print(wsi_name, "done!")
        torch.save(data, f'{output_path}.pt')

### 1.5 Dataset Split

Based on CLAM's `create_split_seq.py`, some changes have been made, the input of the label file has been added, and the task has been set to any number of classifications，re-write the `create_split_seq_re.py` file. The `label.csv` needs to include three columns of useful information: case_id, slide_id and label. 

##### (1) Parameter Description:
- **`--label_path`**: Path of the label file for typing/stage classification task.
- **`--seed`**: Random seed.
- **`--task`**: Task name.
- **`--k`**: Number of splits.
- **`--label_frac`**: Fraction of labels.
- **`--val_frac`**: Fraction of labels for validation.
- **`--test_frac`**: Fraction of labels for test.
- **`--save_dir`**: Saving directory for the split result.

##### (2) Usage Examples:
The command line for dataset split can be structured as follows:

```bash
python create_splits_seq_re.py --label_path /path/to/WSI_bm40/csv/label.csv --seed 1 --task WSI_bm40 --label_frac 1.0 --k 10 --val_frac 0.2 --test_frac 0.2 --save_dir /path/to/WSI_bm40/
```

**Note**: Please replace the paths and values with those relevant to your setup.

## 2. GPU Training:

In `main.py` and `utils/core_utils.py`, some minor additions and changes were made to enable the training of the HIGT model, and re-write the `main_re.py` and `utils/core_utils_re.py` files.

##### (1) Parameter Description:
- **`--max_epochs:`**: Maximum number of epochs to train.
- **`--label_path:`**: The path of the classification label.
- **`--drop_out`**: Enable dropout (p=0.25).
- **`--early_stopping`**: Enable early stopping.
- **`--lr`**: Learning rate.
- **`--k`**:Number of folds.
- **`--label_frac`**: Fraction of training labels.
- **`--exp_code`**: Experiment code for saving results.
- **`--weighted_sample`**: Enable weighted sampling.
- **`--bag_loss`**: Slide-level classification loss function.
- **`--task`**: Task name.
- **`--split_dir`**: Manually specify the set of splits to use.
- **`--model_type`**: Model type used for training.
- **`--log_data`**: Log data using tensorboard.
- **`--data_root_dir`**: Data directory.
- **`--results_dir:`**: Desults directory.

##### (2) Usage Examples:

The command line for GPU training can be structured as follows:
```bash
CUDA_VISIBLE_DEVICES=0 python main_re.py --max_epochs 100 --label_path /path/to/exp/WSI_bm40/csv/label.csv --drop_out --early_stopping --lr 2e-4 --k 10 --label_frac 1.0 --exp_code WSI_bm40 --weighted_sample --bag_loss ce --task WSI_bm40 --split_dir /path/to/exp/WSI_bm40/splits/WSI_bm40_{label_frac*100 --model_type HIGT --log_data --data_root_dir /path/to/exp/WSI_bm40/higt_graph --results_dir /path/to/exp/WSI_bm40/higt_result 
```

## 3. Testing and Evaluation:


In `eval.py`, some minor additions and changes were made to enable the testing and evaluating of the HIGT model, and re-write the `eval_re.py` file.

##### (1) Parameter Description:
- **`--drop_out`**: Enable dropout (p=0.25).
- **`--k`**: Number of folds.
- **`--models_exp_code`**: Fraction of training labels.
- **`--save_exp_code`**: Experiment code for saving results.
- **`--task`**: Task name.
- **`--model_type`**: Model type used for training.
- **`--results_dir:`**: Results directory.
- **`--split_dir`**: Manually specify the set of splits to use.
- **`--data_root_dir`**: Data directory.

##### (2) Usage Examples:

The command line for testing and evaluation can be structured as follows:

```bash
CUDA_VISIBLE_DEVICES=0 python eval_re.py --drop_out --k 10 --models_exp_code WSI_bm40 --save_exp_code WSI_bm40 --task WSI_bm40 --model_type MulGT --results_dir /path/to/exp/WSI_bm40/higt_result --split_dir /path/to/exp/WSI_bm40/splits/WSI_bm40_{label_frac*100} --data_root_dir /path/to/exp/WSI_bm40/higt_graph
```

## 4. Folder Structure Overview:

The final folder structure and contents will look like this:

```
path_to/exp/
    ├── WSI_bm40
        ├── WSI_40
            ├── slide_1.svs
            ├── slide_2.svs
            └── ...
        ├── csv
            ├── bm.csv
            ├── typing/stage_label.csv
            ├── pl_mag10x_patch512_2048.csv
            └── pl_mag5x_patch512_4096.csv
        ├── segmented_patch
            ├── mag10x_patch512_2048
                ├── masks
                    ├── slide_1_masks.jpg
                    ├── slide_2_masks.jpg
                    └── ...
                ├── patches
                    ├── slide_1_patches.h5
                    ├── slide_2_patches.h5
                    └── ...
                ├── stitches
                    ├── slide_1_stitches.jpg
                    ├── slide_2_stitches.jpg
                    └── ...
                └── process_list_autogen.csv
            ├── mag5x_patch512_4096
                ├── masks
                    ├── slide_1_masks.jpg
                    ├── slide_2_masks.jpg
                    └── ...
                ├── patches
                    ├── slide_1_patches.h5
                    ├── slide_2_patches.h5
                    └── ...
                ├── stitches
                    ├── slide_1_stitches.jpg
                    ├── slide_2_stitches.jpg
                    └── ...
                └── process_list_autogen.csv
        ├── extracted_feature
            ├── mag10x_patch512_2048
                ├── h5_files
                    ├── slide_1_feats.h5
                    ├── slide_2_feats.h5
                    └── ...
                ├── pt_files
                    ├── slide_1_feats.pt
                    ├── slide_2_feats.pt
                    └── ...      
            ├── mag5x_patch512_4096
                ├── h5_files
                    ├── slide_1_feats.h5
                    ├── slide_2_feats.h5
                    └── ...
                ├── pt_files
                    ├── slide_1_feats.pt
                    ├── slide_2_feats.pt
                    └── ...      
            ├── thumbnail
                ├── h5_files
                    ├── slide_1_feats.h5
                    ├── slide_2_feats.h5
                    └── ...
                ├── pt_files
                    ├── slide_1_feats.pt
                    ├── slide_2_feats.pt
                    └── ...      
        ├── higt_graph
            ├── pt_files
                ├── slide_1_graph.pt
                ├── slide_2_graph.pt
                └── ...
        ├── splits
            ├── WSI_bm40_{label_frac*100}
                ├── split_0_bool.csv
                ├── split_0_descriptor.csv
                ├── split_0.csv
                └── ... 
        ├── higt_results
            ├── split_0.csv
            ├── s_0_checkpoint.pt
            ├── split_0_results.pkl
            └── ...
```

**Note**: Please replace the paths and values with those relevant to your setup.